<a href="https://colab.research.google.com/github/ColinJ69/Farmers_Friend/blob/main/models/Disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from pathlib import Path
import os
import zipfile
import torchvision

In [ ]:
data_path = Path('train')

if not data_path.is_dir():
    data_path.mkdir(exist_ok =True, parents=True)
if os.path.exists('Train.zip'):
  with zipfile.ZipFile("Train.zip", "r") as zip_ref:
    zip_ref.extractall(data_path)


In [ ]:
transform = torchvision.transforms.Compose([transforms.Resize((640,640)), transforms.ToTensor()])
image_folder = datasets.ImageFolder(root=data_path / 'Train', transform=transform,target_transform=None)
image_dataloader = DataLoader(image_folder, num_workers=2, batch_size=32, shuffle=True)
class_names = image_folder.classes
print(class_names)

['Healthy', 'Powdery', 'Rust']


In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(), nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
    self.conv2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(), nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2))
    self.classifier = nn.Sequential(nn.Dropout(), nn.Linear(3276800, len(class_names)))

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

model = Model()

In [ ]:
from tqdm.auto import tqdm

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.6)
def train():
  epochs = 10
  model.train()
  for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    for batch, (X,y) in enumerate(image_dataloader):
      optimizer.zero_grad()
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      loss.backward()
      optimizer.step()
      epoch_loss += loss
    epoch_loss /= len(image_dataloader)
    print(f' LOSS: {epoch_loss}')


In [ ]:
train()

  0%|          | 0/10 [00:00<?, ?it/s]

 LOSS: 1.0928832292556763
 LOSS: 1.0503745079040527
 LOSS: 0.9176754951477051
 LOSS: 0.7368153929710388
 LOSS: 0.6270769238471985
 LOSS: 0.599634051322937
 LOSS: 0.5363337397575378
 LOSS: 0.5423561334609985
 LOSS: 0.5166577696800232
 LOSS: 0.43054893612861633


In [43]:
torch.save(model.state_dict(), 'disease_model2.pth')